In [1]:
import matplotlib.pyplot as plt
import torch
from torch import nn
import numpy as np 
device = "cuda" if torch.cuda.is_available() else "cpu"

# Import torch.nn.functional as F
import torch.nn.functional as F
import os

# Set your API key here
os.environ["WANDB_API_KEY"] = "5e8d06e5ae60a39c7945e42203c69152c47dbb61"


In [2]:
!pip install transformers
!pip install rouge-score
!pip install sacrebleu


  Preparing metadata (setup.py) ... - done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=207834864c0cd00f823ba39803e9d314aaf77dbf9e6c0adbb32fdb53733534ff
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 5.3 MB/s eta 0:00:00


In [3]:
data_path = "/kaggle/input/pcmag-dataset/train.csv"
test_path="/kaggle/input/pcmag-dataset/test.csv"
valid_path="/kaggle/input/pcmag-dataset/valid.csv"

In [4]:
output_dir_positive = "/kaggle/working/checkpoints/negative"

In [5]:
import pandas as pd
data = pd.read_csv(data_path)
df = pd.DataFrame(data).fillna("NONE")
print(df.columns)
data = pd.read_csv(test_path)
tf = pd.DataFrame(data).fillna("NONE")
data = pd.read_csv(valid_path)
vf = pd.DataFrame(data).fillna("NONE")

Index(['product_name', 'overall_rating', 'review_text', 'positive_comment',
       'negative_comment', 'neural_comment'],
      dtype='object')


In [6]:
# Convert the 'price' column to numeric, invalid parsing will be set as NaN
def is_numeric(value):
    try:
        float(value)
        return True
    except ValueError:
        return False 
df=df.fillna('None')
df = df[df["overall_rating"].apply(is_numeric)]
tf=tf[tf["overall_rating"].apply(is_numeric)]
vf=vf[vf["overall_rating"].apply(is_numeric)]


In [7]:
df['overall_rating'].unique()

array([4. , 3.5, 3. , 4.5, 5. , 2.5, 2. , 1.5, 1. , 0. ])

In [8]:
from sklearn.preprocessing import OneHotEncoder
possible_ratings = [i * 0.5 for i in range(11)] 
encoder = OneHotEncoder(sparse=False)
encoded_df = pd.get_dummies(df['overall_rating'], prefix='rating').reindex(columns=[f'rating_{r}' for r in possible_ratings], fill_value=0)
df = pd.concat([df, encoded_df], axis=1)
encoded_tf = pd.get_dummies(tf['overall_rating'], prefix='rating').reindex(columns=[f'rating_{r}' for r in possible_ratings], fill_value=0)
tf = pd.concat([tf, encoded_tf], axis=1)

# For vf (validation data)
encoded_vf = pd.get_dummies(vf['overall_rating'], prefix='rating').reindex(columns=[f'rating_{r}' for r in possible_ratings], fill_value=0)
vf = pd.concat([vf, encoded_vf], axis=1)

In [9]:
print(df.columns)
print(vf.columns)
print(vf.columns)

Index(['product_name', 'overall_rating', 'review_text', 'positive_comment',
       'negative_comment', 'neural_comment', 'rating_0.0', 'rating_0.5',
       'rating_1.0', 'rating_1.5', 'rating_2.0', 'rating_2.5', 'rating_3.0',
       'rating_3.5', 'rating_4.0', 'rating_4.5', 'rating_5.0'],
      dtype='object')
Index(['product_name', 'overall_rating', 'review_text', 'positive_comment',
       'negative_comment', 'neural_comment', 'rating_0.0', 'rating_0.5',
       'rating_1.0', 'rating_1.5', 'rating_2.0', 'rating_2.5', 'rating_3.0',
       'rating_3.5', 'rating_4.0', 'rating_4.5', 'rating_5.0'],
      dtype='object')
Index(['product_name', 'overall_rating', 'review_text', 'positive_comment',
       'negative_comment', 'neural_comment', 'rating_0.0', 'rating_0.5',
       'rating_1.0', 'rating_1.5', 'rating_2.0', 'rating_2.5', 'rating_3.0',
       'rating_3.5', 'rating_4.0', 'rating_4.5', 'rating_5.0'],
      dtype='object')


In [10]:
# Define custom dataset class
class Custom_Dataset(torch.utils.data.Dataset):
    def __init__(self, data,pgen,ngen,nut_gen, labels):
        self.data = data
        self.positive_gen=pgen
        self.negative_gen=ngen
        self.neutral_gen=nut_gen
        self.labels = labels
        
    def __getitem__(self, index: int) -> (torch.Tensor,torch.Tensor,torch.Tensor,torch.Tensor, list):
        x = self.data[index]
        pgen=self.positive_gen
        ngen=self.negative_gen
        nut_gen=self.neutral_gen
        y = self.labels[index]
        return (x,pgen,ngen,nut_gen, y)
    
    def __len__(self) -> int:
        return len(self.data)

In [11]:
from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(df[['review_text']], df[['overall_rating', 'positive_comment',
#        'negative_comment', 'neural_comment', 'overall_rating_0.0',
#        'overall_rating_1.0', 'overall_rating_1.5', 'overall_rating_2.0',
#        'overall_rating_2.5', 'overall_rating_3.0', 'overall_rating_3.5',
#        'overall_rating_4.0', 'overall_rating_4.5', 'overall_rating_5.0']], test_size=0.2, shuffle=True)

# # Define train and test datasets and dataloaders
# # data,pgen,ngen,nut_gen, labels
train_data = Custom_Dataset(data=df['review_text'],pgen=df['positive_comment'],ngen=df['negative_comment'],nut_gen=df['neural_comment'],labels=df[[ 'rating_0.0', 'rating_0.5',
       'rating_1.0', 'rating_1.5', 'rating_2.0', 'rating_2.5', 'rating_3.0',
       'rating_3.5', 'rating_4.0', 'rating_4.5', 'rating_5.0']])
test_data = Custom_Dataset(data=tf['review_text'],pgen=tf['positive_comment'],ngen=tf['negative_comment'],nut_gen=tf['neural_comment'],labels=tf[['rating_0.0', 'rating_0.5',
       'rating_1.0', 'rating_1.5', 'rating_2.0', 'rating_2.5', 'rating_3.0',
       'rating_3.5', 'rating_4.0', 'rating_4.5', 'rating_5.0']])
valid_data = Custom_Dataset(data=vf['review_text'],pgen=vf['positive_comment'],ngen=vf['negative_comment'],nut_gen=vf['neural_comment'],labels=vf[['rating_0.0', 'rating_0.5',
       'rating_1.0', 'rating_1.5', 'rating_2.0', 'rating_2.5', 'rating_3.0',
       'rating_3.5', 'rating_4.0', 'rating_4.5', 'rating_5.0']])
train_loader = torch.utils.data.DataLoader(dataset=train_data, batch_size=3, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_data, batch_size=1, shuffle=False)
test_loader = torch.utils.data.DataLoader(dataset=valid_data, batch_size=1, shuffle=False)

In [12]:
import transformers
from datasets import Dataset
from datasets import load_dataset, load_metric, load_from_disk
metric = load_metric('rouge',trust_remote_code=True)
model_checkpoints = 'facebook/bart-large-xsum'
# Check if CUDA is available
device = "cuda" if torch.cuda.is_available() else "cpu"



/tmp/ipykernel_24/3103988912.py:4: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric('rouge',trust_remote_code=True)


In [13]:
from transformers import BartTokenizer, BartForConditionalGeneration, Trainer, TrainingArguments
from datasets import Dataset
import torch

# Check if CUDA is available
device = "cuda" if torch.cuda.is_available() else "cpu"
tokenizer = transformers.AutoTokenizer.from_pretrained(model_checkpoints)
def preprocess_data(df):
  #get the dialogue text
  inputs = [dialogue for dialogue in df['review_text']]
  #tokenize text
  model_inputs = tokenizer(inputs,  max_length=1024, padding='max_length', truncation=True)

  #tokenize labels
  with tokenizer.as_target_tokenizer():
    targets = tokenizer(df['negative_comment'], max_length=512, padding='max_length', truncation=True)
    
  model_inputs['labels'] = targets['input_ids']
  #reuturns input_ids, attention_masks, labels
  return model_inputs


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.51k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [14]:
train_df=Dataset.from_pandas(df)
valid_df=Dataset.from_pandas(vf)
test_df=Dataset.from_pandas(tf)
tokenize_train = train_df.map(preprocess_data, batched = True, remove_columns=[ 'review_text', 'negative_comment'])
tokenize_valid = valid_df.map(preprocess_data, batched = True, remove_columns=[ 'review_text', 'negative_comment'])
tokenize_test = test_df.map(preprocess_data, batched = True, remove_columns=[ 'review_text', 'negative_comment'])

Map:   0%|          | 0/13741 [00:00<?, ? examples/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:4126: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/1735 [00:00<?, ? examples/s]

Map:   0%|          | 0/1718 [00:00<?, ? examples/s]

In [15]:
tokenize_train.save_to_disk("./tokenized_dataset/train/checkpoint")
tokenize_valid.save_to_disk("./tokenized_dataset/valid/checkpoint")
tokenize_test.save_to_disk("./tokenized_dataset/test/checkpoint")
# tokenized_train = load_from_disk("./tokenized_dataset/train/checkpoint")
# tokenized_valid = load_from_disk("./tokenized_dataset/valid/checkpoint")
# tokenized_test = load_from_disk("./tokenized_dataset/test/checkpoint")

Saving the dataset (0/1 shards):   0%|          | 0/13741 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1735 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1718 [00:00<?, ? examples/s]

In [16]:
model = transformers.AutoModelForSeq2SeqLM.from_pretrained(model_checkpoints).to(device)
#collator to create batches. It preprocess data with the given tokenizer
collator = transformers.DataCollatorForSeq2Seq(tokenizer, model=model)

pytorch_model.bin:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/309 [00:00<?, ?B/s]

In [17]:
# print(df.columns)
# df["negative_comment"].head()
def compute_rouge(pred):
  predictions, labels = pred
  #decode the predictions
  decode_predictions = tokenizer.batch_decode(predictions, skip_special_tokens=True)
  #decode labels
  decode_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

  #compute results
  res = metric.compute(predictions=decode_predictions, references=decode_labels, use_stemmer=True)
  #get %
  res = {key: value.mid.fmeasure * 100 for key, value in res.items()}

  pred_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
  res['gen_len'] = np.mean(pred_lens)

  return {k: round(v, 4) for k, v in res.items()}

In [18]:
args = transformers.Seq2SeqTrainingArguments(
    evaluation_strategy='epoch',  # Your current evaluation strategy
    save_strategy='epoch',        # Set save strategy to match evaluation strategy
    learning_rate=2e-5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=2,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=3,
    predict_with_generate=True,
    eval_accumulation_steps=1,
    fp16=True,
    load_best_model_at_end=True,
    output_dir=output_dir_positive,
    resume_from_checkpoint=True
)

trainer = transformers.Seq2SeqTrainer(
    model, 
    args,
    train_dataset=tokenize_train,
    eval_dataset=tokenize_valid,
    data_collator=collator,
    tokenizer=tokenizer,
    compute_metrics=compute_rouge
)
trainer.train()

model.save_pretrained("./fine_tuned_model_negative")

# Save the tokenizer
tokenizer.save_pretrained("./fine_tuned_model_nagative")

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:488: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: sivamavi212 (sivamaniepics). Use `wandb login --relogin` to force relogin
wandb: wandb version 0.18.3 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.17.7
wandb: Run data is saved locally in /kaggle/workin

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
0,0.102500,0.109209,23.831900,9.848500,20.499900,22.912900,22.714700
2,0.060500,0.106976,27.808000,12.718200,23.479100,26.623300,25.904300


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 62, 'min_length': 11, 'early_stopping': True, 'num_beams': 6, 'no_repeat_ngram_size': 3, 'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 62, 'min_length': 11, 'early_stopping': True, 'num_beams': 6, 'no_repeat_ngram_size': 3, 'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a G

('./fine_tuned_model_nagative/tokenizer_config.json',
 './fine_tuned_model_nagative/special_tokens_map.json',
 './fine_tuned_model_nagative/vocab.json',
 './fine_tuned_model_nagative/merges.txt',
 './fine_tuned_model_nagative/added_tokens.json',
 './fine_tuned_model_nagative/tokenizer.json')